In [ ]:
import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
import pandas as pd
import numpy as np

df_obs = pd.read_sql('select encounter_id from `obs`', con = conn)
len(df_obs.index)

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

df_labevents_mimic_num = pd.read_sql("SELECT subject_id, value, hadm_id, itemid, storetime from mimic4.labevents where hadm_id IS NOT NULL AND value IS NOT NULL AND value NOT regexp '[[:alpha:]]'", con = mimicconn)

In [ ]:
df_labevents_mimic_num

In [ ]:
# df_labevents_mimic_coded = pd.read_sql("SELECT subject_id, value, hadm_id, itemid, storetime from mimic4.labevents where hadm_id IS NOT NULL AND value IS NOT NULL AND value regexp '[[:alpha:]]'", con = mimicconn)
# df_labevents_mimic_coded

In [ ]:
df_labitems_mimic_ = pd.read_sql("SELECT itemid, label from mimic4.d_labitems where itemid IS NOT NULL AND label IS NOT NULL", con = mimicconn)

In [ ]:
df_labitems_mimic_

In [ ]:
df_labitems_mimic_['label'] = df_labitems_mimic_['label'].str.lower()

In [ ]:
dict_ = dict(zip(df_labitems_mimic_.itemid, df_labitems_mimic_.label))
dict_

In [ ]:
# First dealing numeric labevents

In [ ]:
df_labevents_mimic_num['label'] = df_labevents_mimic_num['itemid'].map(dict_)
df_labevents_mimic_num

In [ ]:
df_labevents_mimic_num['label'].isnull().sum()

In [ ]:
concept_lab_num = pd.read_sql('select * from concept where class_id = 1', con=conn)
concept_lab_num 

In [ ]:
dict_concept_num = dict(zip(concept_lab_num.description, concept_lab_num.concept_id))
len(dict_concept_num)

In [ ]:
df_labevents_mimic_num['concept_id'] = df_labevents_mimic_num['label'].map(dict_concept_num)
df_labevents_mimic_num

In [ ]:
df_labevents_mimic_num.isnull().sum()

In [ ]:
df_labevents_mimic_num = df_labevents_mimic_num.rename({"subject_id": "person_id", "hadm_id": "encounter_id", "storetime": "obs_datetime", "value":"value_numeric" }, axis='columns')

df_labevents_mimic_num = df_labevents_mimic_num.drop(columns=['itemid', 'label'])
df_labevents_mimic_num['date_created']= '2022-01-01'


In [ ]:
import uuid
df_labevents_mimic_num['uuid'] = [uuid.uuid4() for x in range(len(df_labevents_mimic_num))]
df_labevents_mimic_num['creator'] = 1

In [ ]:
column_names = ['person_id', 'concept_id', 'encounter_id', 'obs_datetime', 'value_numeric', 'creator', 'date_created', 'uuid']

df_labevents_mimic_num = df_labevents_mimic_num.reindex(columns=column_names)

In [ ]:
#dealing with nonnumeric labevents

In [ ]:
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded['label'] = df_labevents_mimic_coded['itemid'].map(dict_)
df_labevents_mimic_coded

In [ ]:
concept_lab_coded = pd.read_sql('select * from concept where class_id = 1 and datatype_id=2', con=conn)
concept_lab_coded

In [ ]:
dict_concept_coded = dict(zip(concept_lab_coded.description, concept_lab_coded.concept_id))
len(dict_concept_coded)

In [ ]:
df_labevents_mimic_coded['concept_id'] = df_labevents_mimic_coded['label'].map(dict_concept_coded)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.isnull().sum()

In [ ]:
df_labevents_mimic_coded = df_labevents_mimic_coded.rename({"subject_id": "person_id", "hadm_id": "encounter_id", "storetime": "obs_datetime", "value":"value_text" }, axis='columns')

df_labevents_mimic_coded = df_labevents_mimic_coded.drop(columns=['itemid', 'label'])
df_labevents_mimic_coded['date_created']= '2022-01-01'
import uuid
df_labevents_mimic_coded['uuid'] = [uuid.uuid4() for x in range(len(df_labevents_mimic_coded))]
df_labevents_mimic_coded['creator'] = 1
df_labevents_mimic_coded


In [ ]:
df_labevents_mimic_coded_copy = df_labevents_mimic_coded.copy()
df_labevents_mimic_coded_copy

In [ ]:
conceptid_answer = pd.read_sql('select * from concept where class_id = 11 and datatype_id=4', con=conn)
conceptid_answer

In [ ]:
dict_answer_conceptid = dict(zip(conceptid_answer.description, conceptid_answer.concept_id))
dict_answer_conceptid

In [ ]:
_df_ = pd.read_csv('non-numeric-labevents_.csv')

_df_= _df_[['value', 'value1']]
_df_.dropna(inplace = True)
_df_

In [ ]:
list_of_value1 = set(_df_['value1'].to_list())
list_of_value1

In [ ]:
Removing numerics
b = [item for item in list_of_value1 if not item.isdigit()]
b.remove('274.9')

for index, row in _df_.iterrows():
    if row['value1'] not in b:
        _df_=_df_.drop(index)

_df_      

In [ ]:
dict_values = dict(zip(_df_.value, _df_.value1))
dict_values

In [ ]:
df_labevents_mimic_coded['value_text'] = df_labevents_mimic_coded['value_text'].map(dict_values)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.isnull().sum()

In [ ]:
df_labevents_mimic_coded['value_coded'] = df_labevents_mimic_coded['value_text'].map(dict_answer_conceptid)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_coded.isnull().sum()

In [ ]:
df_concept_answer = pd.read_sql('select * from `concept_answer`', con = conn)
df_concept_answer

In [ ]:
df_concept_answer['concept_id'].nunique()

In [ ]:
dict_answerconcept_coded = dict(zip(df_concept_answer.concept_id, df_concept_answer.answer_concept))
dict_answerconcept_coded

In [ ]:
df_labevents_mimic_coded['value_coded'] = df_labevents_mimic_coded['concept_id'].map(dict_answerconcept_coded)
df_labevents_mimic_coded

In [ ]:
df_labevents_mimic_num_copy = df_labevents_mimic_num.copy()

In [ ]:
df_labevents_mimic_num_copy

In [ ]:
# first 10 million

In [ ]:
df_Obs_labevents_part1 = df_labevents_mimic_num_copy[:10000000]

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x


In [ ]:
df_Obs_labevents_part1.value_numeric = df_Obs_labevents_part1.value_numeric.apply(split)

In [ ]:
import re
def remove_chars(x):
    return re.sub("[^.0-9]+", "", x)
    
df_Obs_labevents_part1.value_numeric = df_Obs_labevents_part1.value_numeric.apply(remove_chars)

In [ ]:
df_Obs_labevents_part1['value_numeric'] = df_Obs_labevents_part1['value_numeric'].replace(r'^\s*$', np.nan, regex=True)
df_Obs_labevents_part1['value_numeric'] = df_Obs_labevents_part1['value_numeric'].fillna(0)


In [ ]:
df_Obs_labevents_part1['value_numeric']=pd.to_numeric(df_Obs_labevents_part1.value_numeric, errors='coerce')

In [ ]:
df_Obs_labevents_part1 = df_Obs_labevents_part1.replace({np.NAN: None})

In [ ]:
df_Obs_labevents_part1

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part1.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part1.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
# Second 10 million

In [ ]:
df_Obs_labevents_part2 = df_labevents_mimic_num_copy[10000000:20000000]

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x

In [ ]:
df_Obs_labevents_part2.value_numeric = df_Obs_labevents_part2.value_numeric.apply(split)

In [ ]:
import re
def remove_chars(x):
    return re.sub("[^.0-9]+", "", x)
    
df_Obs_labevents_part2.value_numeric = df_Obs_labevents_part2.value_numeric.apply(remove_chars)

In [ ]:
df_Obs_labevents_part2['value_numeric'] = df_Obs_labevents_part2['value_numeric'].replace(r'^\s*$', np.nan, regex=True)
df_Obs_labevents_part2['value_numeric'] = df_Obs_labevents_part2['value_numeric'].fillna(0)


In [ ]:
df_Obs_labevents_part2['value_numeric']=pd.to_numeric(df_Obs_labevents_part2.value_numeric, errors='coerce')

In [ ]:
df_Obs_labevents_part2 = df_Obs_labevents_part2.replace({np.NAN: None})

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part2.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part2.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
# Dealing with third 10 million

In [ ]:
df_Obs_labevents_part3 = df_labevents_mimic_num_copy[20000000:30000000]

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x

In [ ]:
df_Obs_labevents_part3.value_numeric = df_Obs_labevents_part3.value_numeric.apply(split)

In [ ]:
import re
def remove_chars(x):
    return re.sub("[^.0-9]+", "", x)
    
df_Obs_labevents_part3.value_numeric = df_Obs_labevents_part3.value_numeric.apply(remove_chars)

In [ ]:
df_Obs_labevents_part3['value_numeric'] = df_Obs_labevents_part3['value_numeric'].replace(r'^\s*$', np.nan, regex=True)
df_Obs_labevents_part3['value_numeric'] = df_Obs_labevents_part3['value_numeric'].fillna(0)


In [ ]:
df_Obs_labevents_part3['value_numeric']=pd.to_numeric(df_Obs_labevents_part3.value_numeric, errors='coerce')

In [ ]:
df_Obs_labevents_part3 = df_Obs_labevents_part3.replace({np.NAN: None})

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part3.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part3.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
# Fourth 10 million 

In [ ]:
df_Obs_labevents_part4 = df_labevents_mimic_num_copy[30000000:40000000]

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x
    
df_Obs_labevents_part4.value_numeric = df_Obs_labevents_part4.value_numeric.apply(split)    

In [ ]:
import re
def remove_chars(x):
    return re.sub("[^.0-9]+", "", x)
    
df_Obs_labevents_part4.value_numeric = df_Obs_labevents_part4.value_numeric.apply(remove_chars)

In [ ]:
df_Obs_labevents_part4['value_numeric'] = df_Obs_labevents_part4['value_numeric'].replace(r'^\s*$', np.nan, regex=True)
df_Obs_labevents_part4['value_numeric'] = df_Obs_labevents_part4['value_numeric'].fillna(0)

In [ ]:
df_Obs_labevents_part4['value_numeric']=pd.to_numeric(df_Obs_labevents_part4.value_numeric, errors='coerce')

In [ ]:
df_Obs_labevents_part4 = df_Obs_labevents_part4.replace({np.NAN: None})

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part4.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part4.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
# last 10 million

In [ ]:
df_Obs_labevents_part5 = df_labevents_mimic_num_copy[40000000:59215010]

In [ ]:
def split(x):
    if '-' in x:
        return x.split('-')[1]
    else:
        return x
    
df_Obs_labevents_part5.value_numeric = df_Obs_labevents_part5.value_numeric.apply(split)    

In [ ]:
import re
def remove_chars(x):
    return re.sub("[^.0-9]+", "", x)
    
df_Obs_labevents_part5.value_numeric = df_Obs_labevents_part5.value_numeric.apply(remove_chars)

In [ ]:
df_Obs_labevents_part5['value_numeric'] = df_Obs_labevents_part5['value_numeric'].replace(r'^\s*$', np.nan, regex=True)
df_Obs_labevents_part5['value_numeric'] = df_Obs_labevents_part5['value_numeric'].fillna(0)

In [ ]:
df_Obs_labevents_part5['value_numeric']=pd.to_numeric(df_Obs_labevents_part5.value_numeric, errors='coerce')

In [ ]:
df_Obs_labevents_part5 = df_Obs_labevents_part5.replace({np.NAN: None})

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part5.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part5.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
df_Obs_labevents_part6 = df_labevents_mimic_num[50000000:60467496]
df_Obs_labevents_part6

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_Obs_labevents_part1.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_Obs_labevents_part1.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 